In [10]:
import tushare as ts
import pandas as pd
import time
from tqdm import tqdm
pro = ts.pro_api('5d2dd2c56bb822ac0e818aaa4b0b344f95c7d5e7b9c83c69e2ed90ff')

In [11]:
fund_list = pd.read_csv("data/fund_list.csv", index_col=0)

In [12]:
failed_times = 0
#success_times = 0
def get_protfolio(fund_code, failed_times, success_times, start_date='20180101', end_date='20201231'):
    for _ in range(3):
        try:
            df = pro.fund_portfolio(ts_code=fund_code, start_date=start_date, end_date=end_date)
            time.sleep(1)
        except:
            time.sleep(1)
        else:
            return df, failed_times
    failed_times += 1
    print(fund_code, "has failed. Total failed times is", failed_times, "success times is", success_times)
    return pd.DataFrame(columns = ['tmp']), failed_times

In [17]:
success = 0
fail = 0
raw_portfolio_list = pd.DataFrame(columns = ['ts_code', 'ann_date', 'end_date', 'symbol', 'mkv'])
for i in tqdm(fund_list['ts_code'], desc="get protfolio"):
    # print("getting the portfolio of fund", i)
    # print(len(get_protfolio(fund_code = i, failed_times = fail, success_times = success)))
    # temp = get_protfolio(fund_code = i, failed_times = fail, success_times = success)
    # fail = temp[-1]
    # df = temp[:-1]
    df, fail = get_protfolio(fund_code = i, failed_times = fail, success_times = success)
    # print(i)
    if not df.empty:
        success += 1
        # print(success)
        if df.iloc[-1]['end_date'][:4] <= '2018':
            df = df.drop(columns=['amount', 'stk_mkv_ratio', 'stk_float_ratio'])
            raw_portfolio_list = pd.concat([raw_portfolio_list, df], ignore_index=True)

get protfolio: 100%|██████████| 6614/6614 [2:09:17<00:00,  1.17s/it]


In [18]:
raw_portfolio_list

,ts_code,ann_date,end_date,symbol,mkv
0,512980.SH,20201028,20200930,600637.SH,7.318376e+07
1,512980.SH,20201028,20200930,002027.SZ,3.056744e+08
2,512980.SH,20201028,20200930,002131.SZ,7.179756e+07
3,512980.SH,20201028,20200930,002555.SZ,1.854357e+08
4,512980.SH,20201028,20200930,002602.SZ,1.048872e+08
...,...,...,...,...,...
1593286,040001.OF,20180122,20171231,601336.SH,5.616000e+07
1593287,040001.OF,20180122,20171231,601699.SH,5.429853e+07
1593288,040001.OF,20180122,20171231,300316.SZ,4.749706e+07
1593289,040001.OF,20180122,20171231,600048.SH,4.733600e+07


In [19]:
raw_portfolio_list.to_csv("data/raw_portfolio_list_whole.csv")